<a href="https://colab.research.google.com/github/CristianCosci/BTC_dataset_Generator_glassnode/blob/main/datasetBuilder_glassnode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests

connected = False

if(not connected):
  drive.mount('/content/drive')
  path = '/content/drive/MyDrive/progettoBTC/'
  connected = True
  f = open("{}secret.txt".format(path))
  API_KEY = f.read().replace("\n", "")
  f.close()

Mounted at /content/drive


In [4]:
def download_dataset(interval, since, dataset_type):
  dati = []
  with open(path+dataset_type,'r') as file:
    # reading each line
    for line in file:
        conta = 0
        # print(line)
        link = line.split()[0]
        riga = {'link': link}
        label = []
        for word in line.split()[1:]:
            label.append(word)
            conta += 1
        for i in range(0, conta, 2):
          riga[label[i]] = label[i+1]
        dati.append(riga)
        riga = {}
  
  df = pd.DataFrame()
  for i in dati:
    link = i.pop('link')
    res = requests.get(link,
    params={'a': 'BTC','s': since,'i': interval, 'api_key': API_KEY})
    foo_df = pd.json_normalize(json.loads(res.text))
    foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
    foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()
    for j in i.keys():
      # print(j)
      df[j] = foo_df[i[j]]
  
  return df

In [5]:
df_hourly = download_dataset(interval = '1h', since = '1356998400', dataset_type = 'hourly_data.txt')
df_daily = download_dataset(interval = '24h', since = '1356998400', dataset_type = 'daily_data.txt')

In [6]:
df_hourly.to_csv(path+"BTC_hourly_dataset.csv")
df_daily.to_csv(path+"BTC_daily_dataset.csv")

In [ ]:
# FOR DEBUG
res = requests.get('https://api.glassnode.com/v1/metrics/indicators/stock_to_flow_ratio' ,
    params={'a': 'BTC','s': '1356998400','i': '24h', 'api_key': API_KEY})
foo_df = pd.json_normalize(json.loads(res.text))
print(foo_df)
foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()